In [1]:
import warnings
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 900)
%autosave 120

Autosaving every 120 seconds


In [2]:
tweets = pd.read_csv(r'.\data\twitter-airline\Tweets.csv', usecols=['text'])
tweets.head()

,text
0,@VirginAmerica What @dhepburn said.
1,@VirginAmerica plus you've added commercials to the experience... tacky.
2,@VirginAmerica I didn't today... Must mean I need to take another trip!
3,"@VirginAmerica it's really aggressive to blast obnoxious ""entertainment"" in your guests' faces &amp; they have little recourse"
4,@VirginAmerica and it's a really big bad thing about it


In [3]:
import re

HANDLE = '@\w+'
LINK = 'https?:t/.co/\w+'
SPECIAL_CHARS = '&lt;|&lt;|&amp;|#'

def clean(text):
    text = re.sub(HANDLE, ' ', text)
    text = re.sub(LINK, ' ', text)
    text = re.sub(SPECIAL_CHARS, ' ', text)
    return text

tweets['text'] = tweets.text.apply(clean)
tweets.head()

,text
0,What said.
1,plus you've added commercials to the experience... tacky.
2,I didn't today... Must mean I need to take another trip!
3,"it's really aggressive to blast obnoxious ""entertainment"" in your guests' faces they have little recourse"
4,and it's a really big bad thing about it


In [4]:
from gensim.parsing.preprocessing import preprocess_string

tweets = tweets.text.apply(preprocess_string).tolist()

In [5]:
from gensim import corpora
from gensim.models.ldamodel import LdaModel

dictionary = corpora.Dictionary(tweets)
corpus = [dictionary.doc2bow(text) for text in tweets]

In [6]:
NUM_TOPICS = 10

ldamodel = LdaModel(corpus,
                    num_topics=NUM_TOPICS,
                    id2word=dictionary,
                    passes=15)

In [7]:
from gensim.models.coherencemodel import CoherenceModel

def calculate_coherence_score(documents, dictionary, model):
    coherence_model = CoherenceModel(model=model,
                                     texts=documents,
                                     dictionary=dictionary,
                                     coherence='c_v')
    return coherence_model.get_coherence()

def get_coherence_values(start, stop):
    for num_topics in range(start, stop):
        ldamodel = LdaModel(corpus,
                            num_topics=num_topics,
                            id2word=dictionary,
                            passes=2)
        coherence = calculate_coherence_score(tweets,
                                              dictionary,
                                              ldamodel)
        yield coherence

In [8]:
min_topics, max_topics = 10, 30

coherence_score = list(get_coherence_values(min_topics, max_topics))

Traceback (most recent call last):
  File "C:\Users\JManuel\anaconda3\envs\ITESO_Anaconda\lib\multiprocessing\queues.py", line 247, in _feed
    send_bytes(obj)
  File "C:\Users\JManuel\anaconda3\envs\ITESO_Anaconda\lib\multiprocessing\connection.py", line 205, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "C:\Users\JManuel\anaconda3\envs\ITESO_Anaconda\lib\multiprocessing\connection.py", line 285, in _send_bytes
    ov, err = _winapi.WriteFile(self._handle, buf, overlapped=True)
BrokenPipeError: [WinError 232] Se está cerrando la canalización
Traceback (most recent call last):
  File "C:\Users\JManuel\anaconda3\envs\ITESO_Anaconda\lib\multiprocessing\queues.py", line 247, in _feed
    send_bytes(obj)
  File "C:\Users\JManuel\anaconda3\envs\ITESO_Anaconda\lib\multiprocessing\connection.py", line 205, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "C:\Users\JManuel\anaconda3\envs\ITESO_Anaconda\lib\multiprocessing\connection.py", line 285, in _se

KeyboardInterrupt: 

In [ ]:
# Prueba con el Modelo LDAMulticore
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamulticore import LdaMulticore

def calculate_coherence_score(documents, dictionary, model):
    coherence_model = CoherenceModel(model=model,
                                     texts=documents,
                                     dictionary=dictionary,
                                     coherence='c_v')
    return coherence_model.get_coherence()

def get_coherence_values(start, stop):
    for num_topics in range(start, stop):
        print(f'Calculating Coherence for {num_topics} topics.')
        lda = LdaMulticore(corpus,
                           num_topics=num_topics,
                           id2word=dictionary,
                           passes=2)
        coherence = calculate_coherence_score(tweets,
                                              dictionary,
                                              lda)
        yield coherence